# **Template for Torch-Pruning**

This template is just built for your convinience.

You are not required to follow the steps and method given below.

In [ ]:
!pip install --upgrade torch_pruning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/410.6 MB 127.8 MB/s eta 0:00:03

In [1]:
import torch
import torchvision
from torchvision.models import mobilenet_v2
import torch_pruning as tp
import torch.nn as nn
import numpy as np
import copy
from matplotlib import pyplot as plt
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

##A Minimal Example ##  
In this section, you will perform channel pruning using the library [Torch-Pruning](https://github.com/VainF/Torch-Pruning).  

The puuner in Torch-Pruning has three main functions: sparse training (optional), importance estimation, and parameter removal.  
Torch-pruning offers two core features to support this process:

tp.importance(): This criteria is utilized to measure the importance of weights.  

tp.pruner(): This is a pruner used for the actual pruning of the parameters.  

For detailed information on this process, please refer to this [tutorial](https://github.com/VainF/Torch-Pruning/wiki/4.-High%E2%80%90level-Pruners/). Additionally, a more practical example is available in [here](https://github.com/VainF/Torch-Pruning/blob/master/benchmarks/main.py).

### 1. Load model


In [2]:
model = torch.load('./mobilenetv2_0.963.pth', map_location="cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [3]:
def recover_model():
  from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
  model = mobilenet_v2()

  NUM_CLASSES=10
  model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
  model = torch.load("mobilenetv2_0.963.pth")
  model.cuda()

  return model

### 2. Prepare a pruner
By default, Torch-Pruning will automatically prune the last non-singleton dim of these parameters. If you want to customize this behaviour, please provide an `unwrapped_parameters` list as the following example.

In [4]:
def train(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  callbacks = None,
  pruner = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
      # Move the data from CPU to GPU
      inputs = inputs.cuda()
      targets = targets.cuda()

      # Reset the gradients (from the last iteration)
      optimizer.zero_grad()

      # Forward inference
      outputs = model(inputs)
      loss = criterion(outputs, targets)

      # Backward propagation
      loss.backward()
      if pruner is not None:
          pruner.regularize(model, reg=1e-5) # for sparsity learning

      # Update optimizer and LR scheduler
      optimizer.step()
      scheduler.step()

      if callbacks is not None:
          for callback in callbacks:
              callback()
  if pruner is not None and isinstance(pruner, tp.pruner.GrowingRegPruner):
      pruner.update_reg() # increase the strength of regularization
      

In [5]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  verbose=True,
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False,
                              disable=not verbose):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Inference
    outputs = model(inputs)

    # Convert logits to class indices
    outputs = outputs.argmax(dim=1)

    # Update metrics
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

In [6]:
transforms = {
    "train": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    "test": Compose([
      Resize((224, 224)),
      ToTensor(),
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
  )

# You can apply your own batch_size
dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=64,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
    drop_last=True
  )

Files already downloaded and verified
Files already downloaded and verified


In [27]:
class MySlimmingPruner(tp.pruner.MetaPruner):
    def regularize(self, model, reg):
        for m in model.modules():
            if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)) and m.affine==True:
                m.weight.grad.data.add_(reg*torch.sign(m.weight.data)) # Lasso for sparsity

In [28]:

class MySlimmingImportance(tp.importance.Importance):
    def __call__(self, group, **kwargs):
        #note that we have multiple BNs in a group, 
        # we store layer-wise scores in a list and then reduce them to get the final results
        group_imp = [] # (num_bns, num_channels) 
        # 1. iterate the group to estimate importance
        for dep, idxs in group:
            layer = dep.target.module # get the target model
            prune_fn = dep.handler    # get the pruning function of target model, unused in this example
            if isinstance(layer, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)) and layer.affine:
                local_imp = torch.abs(layer.weight.data)
                group_imp.append(local_imp)
        if len(group_imp)==0: return None # return None if the group contains no BN layer
        # 2. reduce your group importance to a 1-D scroe vector. Here we use the average score across layers.
        group_imp = torch.stack(group_imp, dim=0).mean(dim=0) 
        return group_imp # (num_channels, )

In [50]:
model = recover_model()

In [7]:
# get the number of group in conv2d or group norm layer in the model
def get_group_num(model):
    group = {}
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.GroupNorm)) and module.groups > 1:
            group[name] = module.groups
    return group
channel_groups = get_group_num(model)

In [51]:
# Importance criterion
# imp = tp.importance.GroupNormImportance(p=1) # or GroupTaylorImportance(), GroupHessianImportance(), etc.
imp = MySlimmingImportance()

# Initialize a pruner with the model and the importance criterion
example_inputs = torch.randn(1, 3, 224, 224).to(device)

ignored_layers = []
for m in model.modules():
  if isinstance(m, torch.nn.Linear) and m.out_features == 10: # ignore the classifier
    ignored_layers.append(m)

iterative_steps = 5

pruner = MySlimmingPruner ( # you can choose any pruner you like.
# pruner = tp.pruner.GroupNormPruner(
    model,
    example_inputs,
    importance=imp,   # Importance Estimator
    # global_pruning=False, # assign uniform pruning ratio to all layers
    pruning_ratio=0.7, # remove 50% channels, ex :ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    iterative_steps = iterative_steps,  # number of steps to achieve the target ch_sparsity.
    ignored_layers = ignored_layers,        # ignore some layers such as the finall linear classifier
    # channel_groups = channel_groups,  # round channels
)

### 3. Prune the model

In [3]:
def getScore(accuracy, mflops):
    return max(0, (200-mflops)/(200-45) - 10*min(max(0.92-accuracy, 0), 1)) * 45

In [52]:
# pretraining
num_pretraining_epoch = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_pretraining_epoch)
criterion = nn.CrossEntropyLoss()

best_accuracy = 0
print(f'Pretraining for sparsity')
for epoch in range(num_pretraining_epoch):
    # At the end of each train iteration, we have to apply the pruning mask
    #    to keep the model sparse during the training
    train(model, dataloader['train'], criterion, optimizer, scheduler, pruner=pruner)
    accuracy = evaluate(model, dataloader['test'])
    is_best = accuracy > best_accuracy
    if is_best:
        best_accuracy = accuracy
    print(f'    Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%')

Pretraining for sparsity


train:   0%|          | 0/781 [00:00<?, ?it/s]

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 1 Accuracy 95.01% / Best Accuracy: 95.01%


train:   0%|          | 0/781 [00:00<?, ?it/s]

...

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 9 Accuracy 96.30% / Best Accuracy: 96.31%


train:   0%|          | 0/781 [00:00<?, ?it/s]

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 10 Accuracy 96.31% / Best Accuracy: 96.31%


In [53]:
# Model size before pruning
base_macs, base_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224).cuda())

model.eval()

# if isinstance(imp, tp.importance.GroupTaylorImportance):
#   # Taylor expansion requires gradients for importance estimation
#   loss = model(example_inputs).sum() # A dummy loss, please replace this line with your loss function and data!
#   loss.backward() # before pruner.step()


# prune
for i in range(iterative_steps):
    pruner.step()  
    # Parameter & MACs Counter
    pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224).cuda())
    MFLOPs = pruned_macs/1e6
    print("The pruned model:")
    print(model)
    print("Summary:")
    print("MFLOPs: ")
    print(MFLOPs)



The pruned model:
MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 27, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(27, 27, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=27, bias=False)
          (1): BatchNorm2d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(27, 13, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(13, 82, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): Ba

In [54]:

accuracy = evaluate(model, dataloader['test'])
print(f"Accuracy before fine-tuning: {accuracy:.2f}%")

# finetune the pruned model here
# finetune(model)
num_finetune_epochs = 50
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_finetune_epochs)
criterion = nn.CrossEntropyLoss()

best_sparse_model_checkpoint = dict()
best_accuracy = 0
print(f'Finetuning Fine-grained Pruned Sparse Model')
for epoch in range(num_finetune_epochs):
    # At the end of each train iteration, we have to apply the pruning mask
    #    to keep the model sparse during the training
    train(model, dataloader['train'], criterion, optimizer, scheduler)
    accuracy = evaluate(model, dataloader['test'])
    is_best = accuracy > best_accuracy
    if is_best:
        best_sparse_model_checkpoint['state_dict'] = copy.deepcopy(model.state_dict())
        best_accuracy = accuracy
    print(f'    Epoch {epoch+1} Accuracy {accuracy:.2f}% / Best Accuracy: {best_accuracy:.2f}%')

# Parameter & MACs Counter
accuracy = evaluate(model, dataloader['test'])
print(f"Accuracy after fine-tuning: {accuracy:.2f}%")

eval:   0%|          | 0/156 [00:00<?, ?it/s]

Accuracy before fine-tuning: 10.02%
Finetuning Fine-grained Pruned Sparse Model


train:   0%|          | 0/781 [00:00<?, ?it/s]

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 1 Accuracy 40.40% / Best Accuracy: 40.40%


...

train:   0%|          | 0/781 [00:00<?, ?it/s]

eval:   0%|          | 0/156 [00:00<?, ?it/s]

    Epoch 50 Accuracy 80.11% / Best Accuracy: 80.49%


eval:   0%|          | 0/156 [00:00<?, ?it/s]

Accuracy after fine-tuning: 80.11%


In [7]:
pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224).cuda())
MFLOPs = pruned_macs/1e6
accuracy = evaluate(model, dataloader['test'])

print("Accuracy: ", accuracy)
print("MFLOPs: ", MFLOPs)

eval:   0%|          | 0/156 [00:00<?, ?it/s]

Accuracy:  80.10816955566406
MFLOPs:  37.99845


In [8]:
accuracy = 80.10816955566406
MFLOPS = 37.99845
score = getScore(accuracy, MFLOPS)
print(f"Score: {score}")

Score: 47.03270806451613


In [33]:

torch.save(model, "mobilenet_37_80.pth")